In [6]:
import matplotlib
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
matplotlib.rcParams['font.sans-serif'] = "KaiTi"
from datetime import date, datetime

## Souhu 数据接口

搜狐数据接口：

http://q.stock.sohu.com/hisHq?code={code}&start={start}&end={end}，{code}替换为股票代码，大陆股票代码加前缀cn_。{start}替换为起始日期，{end}替换为结束日期。


例如：在浏览器地址栏输入：

http://q.stock.sohu.com/hisHq?code=cn_601766,cn_000002&start=20150601&end=20160414

数组中数据一次对应含义如下：

日期,
今日开盘价,
今日收盘价,
涨跌幅度，
涨跌百分比,
今日最低价,
今日最高价，
今日成交股票手数,
今日成交金额(单位：万)，
换手率

In [8]:
import json
url = "https://q.stock.sohu.com/hisHq?code=cn_600036&start=20220101&end=20220723"
r = httpx.get(url)
data = json.loads(r.text)
stock = data[0]['hq']

json_data = StringIO(json.dumps(stock))

df = pd.read_json(json_data, orient="records")
df.columns = ['日期', '开盘价','收盘价','涨跌幅度', '涨跌百分比','最低价', '最高价', '成交股票手数', '成交金额(单位：万)', '换手率']

df = df.set_index("日期")
df.index = pd.to_datetime(df.index)
df

,开盘价,收盘价,涨跌幅度,涨跌百分比,最低价,最高价,成交股票手数,成交金额(单位：万),换手率
日期,,,,,,,,,
2022-07-22,35.55,35.59,0.13,0.37%,35.25,36.05,510333,181747.73,0.25%
2022-07-21,35.54,35.46,-0.39,-1.09%,35.37,35.83,466241,165613.77,0.23%
2022-07-20,35.94,35.85,0.14,0.39%,35.71,36.12,469689,168511.69,0.23%
2022-07-19,35.50,35.71,0.08,0.22%,35.20,35.85,537814,191044.84,0.26%
2022-07-18,35.15,35.63,0.76,2.18%,35.01,35.84,903080,320222.72,0.44%
...,...,...,...,...,...,...,...,...,...
2022-01-10,49.47,49.23,0.18,0.37%,48.91,50.00,570936,281814.50,0.28%
2022-01-07,48.14,49.05,0.90,1.87%,48.04,49.14,660737,321952.34,0.32%
2022-01-06,48.82,48.15,-0.90,-1.83%,48.10,49.39,681867,329982.56,0.33%


## 新浪财经基金净值数据

url：

http://stock.finance.sina.com.cn/fundInfo/api/openapi.php/CaihuiFundInfoService.getNav?symbol=040011&datefrom=20220101&dateto=20220723&page=1

接口是分页的形式，需要遍历 page。

```
'日期' => 'fbrq',
'单位净值（元）' => 'jjjz',
'累计净值（元）' => 'ljjz',
```

In [11]:
page = 1
total = pd.DataFrame([])

while True:
    url = f'http://stock.finance.sina.com.cn/fundInfo/api/openapi.php/CaihuiFundInfoService.getNav?symbol=040011&datefrom=20220101&dateto=20250102&page={page}'
    r = httpx.get(url)
    data = json.loads(r.text)
    result = data["result"]["data"]['data']
    if (len(result) == 0):
        break
    json_data = StringIO(json.dumps(result))
    df = pd.read_json(json_data, orient="records")
    total = pd.concat([total, df])
    page = page + 1


total.columns = ['日期', '单位净值（元）', '累计净值（元）']
total = total.set_index('日期')
total.index = pd.to_datetime(total.index)
total

,单位净值（元）,累计净值（元）
日期,,
2024-12-31,1.8848,3.4148
2024-12-30,1.9280,3.4580
2024-12-27,1.9230,3.4530
2024-12-26,1.9329,3.4629
2024-12-25,1.9181,3.4481
...,...,...
2022-01-10,2.9101,4.4401
2022-01-07,2.8996,4.4296
2022-01-06,2.9124,4.4424


## 新浪财经

新浪财经网页地址：https://vip.stock.finance.sina.com.cn/mkt/#hqIndex

url地址：

```
https://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getHQNodeData?page=1&num=80&sort=symbol&asc=1&node=hs_a&symbol=&_s_r_a=init
```

`node` 参数：hs_a 为沪深A股


In [12]:
url = "https://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getHQNodeData?page=1&num=80&sort=symbol&asc=1&node=hs_a&symbol=&_s_r_a=init"
r = httpx.get(url)
data = json.loads(r.text)
json_data = StringIO(json.dumps(data))
df = pd.read_json(json_data, orient="records")
df = df.set_index("symbol")
df

,code,name,trade,pricechange,changepercent,buy,sell,settlement,open,high,low,volume,amount,ticktime,per,pb,mktcap,nmc,turnoverratio
symbol,,,,,,,,,,,,,,,,,,,
bj430017,430017,星昊医药,13.57,-0.24,-1.738,13.56,13.57,13.81,13.87,13.90,13.50,868372,11913613,13:58:56,16.155,1.088,1.659451e+05,164284.119400,0.71728
bj430047,430047,诺思兰德,11.19,-0.34,-2.949,11.16,11.19,11.53,11.53,11.55,11.11,1295806,14674558,13:58:56,-58.895,8.981,3.069103e+05,201539.091813,0.71947
bj430090,430090,同辉信息,5.32,-0.20,-3.623,5.33,5.35,5.52,5.54,5.55,5.27,4444473,24202982,13:58:56,-8.276,7.152,1.060454e+05,91236.964196,2.59156
bj430139,430139,华岭股份,23.40,-0.78,-3.226,23.41,23.43,24.18,24.01,24.39,23.05,5397572,128232406,13:58:56,83.571,5.736,6.306473e+05,609878.751300,2.07096
bj430198,430198,微创光电,10.23,-0.19,-1.823,10.21,10.24,10.42,10.34,10.58,10.05,2434722,25250814,13:58:56,-8.597,4.313,1.650752e+05,71859.998694,3.46607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bj832982,832982,锦波生物,203.00,-4.00,-1.932,202.33,203.00,207.00,206.29,208.00,202.26,216552,44200756,13:58:56,43.750,13.643,1.796790e+06,931588.924000,0.47188
bj833030,833030,立方控股,15.29,-0.21,-1.355,15.23,15.29,15.50,15.55,15.95,15.00,858694,13376154,13:58:56,31.204,2.468,1.410399e+05,85541.825424,1.53486
bj833075,833075,柏星龙,26.91,0.18,0.673,26.83,26.91,26.73,26.67,28.00,25.84,2185950,59485593,13:58:56,37.848,4.321,1.744172e+05,66262.511250,8.87740


**基金数据：**

url：

```
https://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getFundNetData?page=2&num=40&sort=symbol&asc=1&node=open_fund&_s_r_a=page
```


In [14]:
url = "https://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getFundNetData?page=1&num=80&sort=symbol&asc=1&node=open_fund&_s_r_a=page"
r = httpx.get(url)
data = json.loads(r.text)
json_data = StringIO(json.dumps(data))
df = pd.read_json(json_data, orient="records", dtype={'symbol': str})
df.columns = ["代码", "名称", "最新公布净值", "累计净值", "前单位净值", "净值增长率", "日期", "基金规模份额（亿份）"]
df = df.set_index('代码')
df

,名称,最新公布净值,累计净值,前单位净值,净值增长率,日期,基金规模份额（亿份）
代码,,,,,,,
000001,华夏成长混合A,0.8220,3.3850,0.8360,-1.674641,2024-12-31,30.6684
000003,中海可转债债券A,0.8330,1.0430,0.8420,-1.068884,2024-12-31,0.8822
000004,中海可转债债券C,0.8160,1.0260,0.8250,-1.090909,2024-12-31,0.4549
000005,嘉实增强信用定期债券,1.0350,1.5415,1.0337,0.125762,2024-12-31,2.6893
000006,西部利得量化成长混合A,1.9209,2.2489,1.9543,-1.709052,2024-12-31,4.4580
...,...,...,...,...,...,...,...
000116,嘉实丰益纯债定期债券A,1.0276,1.5443,1.0265,0.107160,2024-12-31,9.9406
000117,广发轮动配置混合,1.9420,1.9420,1.9560,-0.715746,2024-12-31,1.3937
000118,广发聚鑫债券A,1.5255,2.3147,1.5277,-0.144007,2024-12-31,46.2496


**ETF数据：**

url:

```
https://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getFundNetData?page=3&num=40&sort=symbol&asc=1&node=etf_jz_fund&_s_r_a=page
```

In [16]:
url = "https://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getFundNetData?page=1&num=80&sort=symbol&asc=1&node=etf_jz_fund&_s_r_a=page"
r = httpx.get(url)
data = json.loads(r.text)
json_data = StringIO(json.dumps(data))
df = pd.read_json(json_data, orient="records", dtype={'symbol': str})
df.columns = ["代码", "名称", "最新公布净值", "累计净值", "前单位净值", "净值增长率", "日期", "基金规模份额（亿份）"]
df = df.set_index('代码')
df

,名称,最新公布净值,累计净值,前单位净值,净值增长率,日期,基金规模份额（亿份）
代码,,,,,,,
159001,易方达保证金货币A,0.3154,3.4740,0.0000,NaN,2024-12-31,0.0279
159003,招商保证金快线A,0.3939,1.4010,0.0000,NaN,2024-12-31,0.0131
159005,汇添富收益快钱货币A,0.3940,1.4930,0.0000,NaN,2024-12-31,0.0042
159150,易方达深证50ETF,1.1849,1.1849,1.2049,-1.659889,2024-12-31,3.1800
159300,富国沪深300ETF,4.0256,1.1227,4.0912,-1.603442,2024-12-31,3.3486
...,...,...,...,...,...,...,...
159521,平安国证2000ETF,0.9590,0.9590,0.9863,-2.767921,2024-12-31,0.6327
159523,华夏中证智选300成长创新策略ETF,1.0123,1.0123,1.0291,-1.632494,2024-12-31,0.7652
159525,富国中证红利低波动ETF,1.0479,1.0479,1.0561,-0.776442,2024-12-31,1.6874
